In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

TensorFlow 2.x selected.


In [0]:
import pandas as pd 

In [0]:
url = "https://github.com/motazsaad/work-online-ds/raw/master/data/melbourne-housing-market/Melbourne_housing_FULL.csv"

dataframe = pd.read_csv(url)
dataframe.head(2)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0


# Steps: 
# 1. execlude obj attributes
# 2. split data
# 3. load as tensor slice data
# 4. shuffle and batch 
# 5. apply sequential model

In [0]:
dataframe = dataframe.select_dtypes(exclude=['object']).dropna(axis=0)

In [0]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


5687 train examples
1422 validation examples
1778 test examples


In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target, shuffle=True, batch_size=100):
  dataframe = dataframe.copy()
  labels = dataframe.pop(target)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 100 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(dataframe=train, target='Price', shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(dataframe=val,  target='Price', shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(dataframe=test,  target='Price', shuffle=False, batch_size=batch_size)

In [0]:
dataframe.keys()

Index(['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')

In [0]:
feature_columns = []

# numeric cols
for col in dataframe.keys():
  if col == 'Price':
    continue
  feature_columns.append(feature_column.numeric_column(col))

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.DenseFeatures(feature_columns))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [0]:
  model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae', 'mse'])

In [0]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=300)

Epoch 1/300
57/57 [==============================] - 0s 8ms/step - loss: 134704306786.8070 - mae: 245476.9219 - mse: 134714875904.0000 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00 - val_mse: 0.0000e+00
Epoch 2/300
57/57 [==============================] - 0s 5ms/step - loss: 134647243114.7352 - mae: 244770.0469 - mse: 134414819328.0000 - val_loss: 217934766626.1333 - val_mae: 271459.4062 - val_mse: 205744078848.0000
Epoch 3/300
57/57 [==============================] - 0s 5ms/step - loss: 133456797470.1861 - mae: 244166.4531 - mse: 133851127808.0000 - val_loss: 216182212198.4000 - val_mae: 263786.5312 - val_mse: 203389829120.0000
Epoch 4/300
57/57 [==============================] - 0s 5ms/step - loss: 136137862029.5673 - mae: 247092.0781 - mse: 135600447488.0000 - val_loss: 223995327829.3333 - val_mae: 257447.2656 - val_mse: 206341275648.0000
Epoch 5/300
57/57 [==============================] - 0s 5ms/step - loss: 135743339253.3315 - mae: 247897.4219 - mse: 135971340288.0000 - val_loss: 

In [0]:
model.evaluate(test_ds)

18/18 [==============================] - 0s 3ms/step - loss: 131648623047.1111 - mae: 243630.3594 - mse: 132271185920.0000


[131648623047.11111, 243630.36, 132271190000.0]